In [1]:
import numpy as np
from numpy import genfromtxt
from itertools import combinations
import torch
import math 
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

### data structuring and retrieving

In [2]:
my_data = genfromtxt('data_stream_2121.txt', delimiter=',')
#my_data[:,4] = my_data[:,4]+4500  #this step is needed, I only want positive time values, so that I can use the time as a radius
my_data[:,4] = 1  #this step is needed, I only want positive time values, so that I can use the time as a radius

# ### structure of mydata : eventnr, energy, theta, phi, hit-time
my_data = my_data*[1.,1.,3.14159/180,3.14159/180,1.]


#### some definitions...

In [3]:
def sph2cart(az, el, r, energy):
    rsin_theta = r*np.sin(el)
    x = rsin_theta*np.cos(az)
    y = rsin_theta*np.sin(az)
    z = r*np.cos(el)
    return x, y, z , energy


def cart2sph(x, y, z, energy):
    hxy = np.hypot(x, y)
    r = np.hypot(hxy, z)
    th = np.arccos(z/r)
    az = np.arctan2(y,x)
    return th, az, r , energy

def mag(u, N):

    # Stores the final magnitude
    magnitude = 0

    # Traverse the array
    for i in range(N):
        magnitude += u[i] * u[i]

    # Return the square root of magnitude
    return math.sqrt(magnitude)


def dotProd(u, v, N):

    # Stores the dot product
    prod = 0

    # Traverse the array
    for i in range(N):
        prod = prod + u[i] * v[i]

    # Return the product
    return prod


def angleVector(u, v, N):

    # Stores the dot product of vectors
    dotProductOfVectors = dotProd(u, v, N)

    # magnitude of vector u
    magOfu = mag(u, N)

    #magnitude of vector v
    magOfv = mag(v, N)
    no_angle_between = 0.01
    # angle between given vectors
    if (dotProductOfVectors/ (magOfu * magOfv) > 0.999):
        return no_angle_between
    else:
        angle = math.acos(dotProductOfVectors
            / (magOfu * magOfv))
        #return the angle [rad]
        return angle


### === end of definitions ===

---

### now the clustering algorithm

In [ ]:
def run_r3b_clustering(clustersize):
    all_events = 0.
    well_reconstructed = 0.
    summed_false_negative = 0.
    summed_false_positive = 0.
    not_reconstructed = 0.
    array_unique_events = np.unique(my_data[:,0])
    gamma_energy_distr = []
    good_counts = 0
    all_counts = len(array_unique_events)
    v_cluster = np.empty((0,4))
    for i in range(0,(len(array_unique_events)-3),3):
    #for i in range(0,9,3):
        E1 = my_data[my_data[:,0] == array_unique_events[i]]
        E2 = my_data[my_data[:,0] == array_unique_events[i+1]]
        E3 = my_data[my_data[:,0] == array_unique_events[i+2]]
        E_1 = E1[:,[2,3,4,1]]
        E_2 = E2[:,[2,3,4,1]]
        E_3 = E3[:,[2,3,4,1]]
        ####transform into cartesian coordinates    
        cart_e1 = sph2cart(E_1[:,1],E_1[:,0],E_1[:,2],E_1[:,3])
        np_cart_e1 = np.asarray(cart_e1)
        np_cart_e1 = np.transpose(np_cart_e1)

        cart_e2 = sph2cart(E_2[:,1],E_2[:,0],E_2[:,2],E_2[:,3])
        np_cart_e2 = np.asarray(cart_e2)
        np_cart_e2 = np.transpose(np_cart_e2)

        cart_e3 = sph2cart(E_3[:,1],E_3[:,0],E_3[:,2],E_3[:,3])
        np_cart_e3 = np.asarray(cart_e3)
        np_cart_e3 = np.transpose(np_cart_e3)
        #create flat arrays for the 3 clusters to compare later with the reconstructed clusters
        arr_energy1 = np_cart_e1[:,3]
        arr_energy1 = arr_energy1.flatten()
        arr_energy2 = np_cart_e2[:,3]
        arr_energy2 = arr_energy2.flatten()
        arr_energy3 = np_cart_e3[:,3]
        arr_energy3 = arr_energy3.flatten()
        list_of_energies = [arr_energy1,arr_energy2,arr_energy3]
        list_of_energies = [l.tolist() for l in list_of_energies]
        #print(list_of_energies)
        all_events += len(list_of_energies)

        anal_list_of_energies = list_of_energies.copy()

        X = np.vstack([np_cart_e1,np_cart_e2,np_cart_e3])
        data = pd.DataFrame(X, columns = ['x','y','z','energy'])
        data= data.to_numpy()
        data = data[data[:,3].argsort()[::-1]]
        arr_reconstructed_clusters = list()
        while (data.shape[0]):
            v_ref = data[0,:]
            v_ref = np.reshape(v_ref,(1,4))
            e_sum = 0
            v_temp = np.empty([0,4])
            arr_single_cluster = list()
            for i in range (data.shape[0]):
                if(i == 0):
                    e_sum += data[0,3]
                    arr_single_cluster.append(data[0,3])
                else:
                    #calculate angle between first entry and the other ones
                    angle_ref_hit = angleVector(v_ref[0:3].flatten(),(data[i,0:3]).flatten(),3)
                    if (angle_ref_hit < clustersize):
                        e_sum += data[i,3]
                        arr_single_cluster.append(data[i,3])
                    else:
                        v_temp = np.append(v_temp,np.array(data[i,:]))
            arr_reconstructed_clusters.append(arr_single_cluster)
            #print(arr_reconstructed_clusters)
            v_temp = np.reshape(v_temp,(-1,4))
            v_ref[0,3] = e_sum
            v_cluster = np.append(v_cluster,v_ref,axis=0)
            data = v_temp
        anal_arr_reconstructed_clusters = arr_reconstructed_clusters.copy()
        print(arr_reconstructed_clusters)
